# Understand and Retrieve your portfolio's credit risk exposures

## Overview
A UK based food manufacturer has excess cash and has set up its own fund to invest into equities and shares. For reporting purposes, this company wants to know for each position in their 15 portfolios, what is its credit exposure? By knowing the price of the [CDS 5Y](https://www.investopedia.com/terms/c/creditdefaultswap.asp), the food manufacturer can determine how much they have to pay to offset the credit risk associated to an instrument within a portfolio. When utilizing LSEG's desktop Portfolio management tools, users have access to a powerful suite of details.  Unfortunately, the credit exposure information is neither available directly within the portfolio nor in any templates of the portfolio reporting tool.

Does that mean it’s not possible to provide this information when the CDS 5Y price is quoted by Refinitiv? Absolutely not! In this article, we will describe how to leverage Python to enrich the portfolio details with the credit risk exposures.

## Getting Started
The following notebook utilizes our Portfolio Analytics desktop capabilities to review and assess your issuer level and portfolios' credit risk exposure. LSEG's extensive content set, coupled with our [LSEG Data Libraries](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python) offers a simple mechanism to retrieve your portfolio details and retrieve current pricing data used to generate desired results.

For convenience, the workflow generates output in an Excel sheet containing the results of our analysis.

#### Learn more

> To learn more about the LSEG Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/tutorials), 
 [Documentation](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

> If you have any questions regarding using the API, please post them on 
the [LSEG Data Q&A Forum](https://community.developers.refinitiv.com/). 
The LSEG Developer Community will be happy to help. 

## Some Imports to start with

In [1]:
# The Refinitiv Data Library for Python
import lseg.data as ld

# Popular container/dataframe to capture and minipulate data we extract
import pandas as pd

# Some basic UI
from refinitiv_widgets import Checkbox
from refinitiv_widgets import ProgressBar

ld.__version__

'2.0.1'

In [2]:
# Uncomment the following to see a larger data set when displaying a dataframe to the screen.
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)

## Open the data session

The open_session() function establishes a data session to retrieve our content within the desktop.

> Note: A desktop session is intentionally defined here to take advantage of some convenient capabilities within the desktop application, such as the ability to use the Portfolio & Lists Manager (PAL) feature.

In [3]:
ld.open_session()

<lseg.data.session.Definition object at 0x7f597f717490 {name='codebook'}>

## Code Setup

It's usually best practice to define constants that are potentially used through the coding segments within your notebook.  A constant is a special type of variable whose value cannot be changed. Using a constant in a script improves its readability, its reusability and its maintainability.

In [4]:
# Maximum number of items a request to get data can retrieve
MAX_ITEMS_PER_REQUEST = 10000

# Fields to request
PORTFOLIO_NAME_FLD = "TR.PortfolioName"
PORTFOLIO_CONSTITUENTS_FLD = "TR.PortfolioConstituentName"
PORTFOLIO_SHARES_FLD = "TR.PortfolioShares"
ASSET_CATEGORY_FLD = "TR.AssetCategory"
CDS_PRIMARY_RIC_FLD = "TR.CDSPrimaryCDSRic"

# Some columns used below by the dataframe
ASSET_CATEGORY_COL = "Asset Category Description"
PRIMARY_CDS_RIC_COL = "Primary CDS RIC"
CLOSE_PRICE_COL = "Close Price"

# Price data
HISTORICAL_CLOSE = "HST_CLOSE"

## Portfolio Definition
At the heart of the following workflow is the ability for users to evaluate their portfolios to determine the credit exposure. For simplicity, we are going to use a pre-built portfolio defining the list of instruments that can be used within our analysis.  

> Note: Users can define their own portfolios by utilizing the power and capabilities of the Portfolio & List Manager service within the desktop.

The following fields are requested to provide a breakdown of the constituents within the portfolio:
1.	<pre>The Name of the Portfolio                        <b>[TR.PortfolioName]</b></pre>
2.	<pre>The issuer Name of the Constituents in Portfolio <b>[TR.PortfolioConstituentName]</b></pre>
3.	<pre>The Number of units of portfolio constituent     <b>[TR.PortfolioShares]</b></pre>
4.	<pre>The full description of the asset category       <b>[TR.AssetCategory]</b></pre>
5.	<pre>The primary CDS RIC for this reference entity    <b>[TR.CDSPrimaryCDSRic]</b></pre>

The 'TR.CDSPrimaryCDSRic' field represents the fundamental data field defining the RIC (Refinitiv Identification Code) of the 5-year Credit Default Swap (CDS 5Y). This field is important for portfolio managers as it allows them to monitor how much credit exposure they have on the securities they are placing their bets on.

### The id of the portfolio.

In [5]:
# The id of the portfolio as defined within the desktop (Eikon/Refinitiv Workspace)
portfolio_id = "SAMPLE_EU_DEV_FI"

### Load the portfolio data...
The <i>LSEG Data Library for Python</i> utilizes the 'Portfolio()' syntax to extract and pull multiple attributes specific to the referenced portfolio, in particular, the credit risk instrument.

In [6]:
# Open the portfolio.  Define the list of fields required for our analysis
portfolio_columns = [PORTFOLIO_NAME_FLD,PORTFOLIO_CONSTITUENTS_FLD,PORTFOLIO_SHARES_FLD,ASSET_CATEGORY_FLD,CDS_PRIMARY_RIC_FLD]

portfolio = f"Portfolio({portfolio_id})"
df = ld.get_data(portfolio, portfolio_columns)
if "Instrument" not in df.columns:
    print(f"Impossible to open the portfolio {portfolio_id}.")

In [7]:
# Display some of the data...
df.head(10)

,Instrument,Portfolio Name,Issuer Name,Portfolio Shares,Asset Category Description,Primary CDS RIC
0,.ONECHF,EUROPE DEVELOPED WITH FIXED INC,Swiss Franc,45872.0,,<NA>
1,.ONEUKP,EUROPE DEVELOPED WITH FIXED INC,U K Pound,37265.0,,<NA>
2,.ONEUSD,EUROPE DEVELOPED WITH FIXED INC,U S Dollar,50000.0,,<NA>
3,.ONEEUR,EUROPE DEVELOPED WITH FIXED INC,Euro,44516.0,,<NA>
4,SOF.BR,EUROPE DEVELOPED WITH FIXED INC,SOFINA SA,27200.0,Ordinary Share,
5,MAERSKb.CO,EUROPE DEVELOPED WITH FIXED INC,AP MOELLER - MAERSK A/S,1700.0,Ordinary Share,MAER5YEUAM=MT
6,DSV.CO,EUROPE DEVELOPED WITH FIXED INC,DSV A/S,140800.0,Ordinary Share,
7,DEMANT.CO,EUROPE DEVELOPED WITH FIXED INC,DEMANT A/S,202600.0,Ordinary Share,
8,GN.CO,EUROPE DEVELOPED WITH FIXED INC,GN STORE NORD A/S,135600.0,Ordinary Share,
9,CAPP.PA,EUROPE DEVELOPED WITH FIXED INC,CAPGEMINI SE,125200.0,Ordinary Share,CPGM5YEUAM=R


### Data Cleansing
In some cases, the referenced portfolio of instruments may include entries not related to CDS-based positions.  As such, we'll need to clean the data by removing all unrelated entries.  Specifically, the rows where the Primary CDS RIC cell is unavailable, i.e. missing values or values filled with <b>&lt;NA&gt;</b>.

For example, the above display shows the first 10 positions within our portfolio where a number of them contain missing values for the Primary CDS RIC.  We are only interested in the positions where a valid RIC is present.

In [8]:
# Remove all rows/entries where we are missing relevant data.  
# To simplify this, we'll first ensure blank values contain <NA>.
df[ASSET_CATEGORY_COL] = df[ASSET_CATEGORY_COL].replace('', pd.NaT)
df[PRIMARY_CDS_RIC_COL] = df[PRIMARY_CDS_RIC_COL].replace('', pd.NaT)

# Now we can simply remove all rows where the following columns contain <NA>
df = df.dropna(subset=[ASSET_CATEGORY_COL, PRIMARY_CDS_RIC_COL], axis=0)

# Remove duplicate CDS RICs
df = df.drop_duplicates(subset=PRIMARY_CDS_RIC_COL, keep="first")

In [9]:
# The results will only contain entries where we have populated data
df.head(10)

,Instrument,Portfolio Name,Issuer Name,Portfolio Shares,Asset Category Description,Primary CDS RIC
5,MAERSKb.CO,EUROPE DEVELOPED WITH FIXED INC,AP MOELLER - MAERSK A/S,1700.0,Ordinary Share,MAER5YEUAM=MT
9,CAPP.PA,EUROPE DEVELOPED WITH FIXED INC,CAPGEMINI SE,125200.0,Ordinary Share,CPGM5YEUAM=R
10,PERP.PA,EUROPE DEVELOPED WITH FIXED INC,PERNOD RICARD SA,207500.0,Ordinary Share,PERP5YEUAM=R
11,PRTP.PA,EUROPE DEVELOPED WITH FIXED INC,KERING SA,78500.0,Ordinary Share,PRTP5YEUAM=R
12,STMPA.PA,EUROPE DEVELOPED WITH FIXED INC,STMICROELECTRONICS NV,712200.0,Ordinary Share,STM5YEUAM=R
13,ALSO.PA,EUROPE DEVELOPED WITH FIXED INC,ALSTOM SA,255534.549639,Ordinary Share,ALSO5YEUAM=R
14,ADSGn.DE,EUROPE DEVELOPED WITH FIXED INC,ADIDAS AG,163600.0,Ordinary Share,ADSG5YEUAM=R
15,DBKGn.DE,EUROPE DEVELOPED WITH FIXED INC,DEUTSCHE BANK AG,797400.0,Ordinary Share,DB5YEUAM=R
16,MRCG.DE,EUROPE DEVELOPED WITH FIXED INC,MERCK KGAA,50500.0,Ordinary Share,MRCG5YEUAM=
20,STLAM.MI,EUROPE DEVELOPED WITH FIXED INC,STELLANTIS NV,940300.0,Ordinary Share,FCAU5YEUAM=R


If you pay attention to the left column value, which represents an index into the table, you can get a sense of specific rows we filtered out.

### Return only price data for bonds?
As a final step to data cleansing, we can optionally choose to process only bonds within our portfolio.  The following UI prompt is a simple way to filter out all positions that are not classified as a bond.

In [10]:
# The final, cleansed data will live within a container called 'assets'
assets = df
bonds = Checkbox(label='Check to retrieve bonds only')
bonds

Checkbox(label='Check to retrieve bonds only', tooltip='')

In [11]:
if bonds.checked:
    assets = assets[assets[ASSET_CATEGORY_COL].str.contains('Bond')].copy()

In [12]:
assets

,Instrument,Portfolio Name,Issuer Name,Portfolio Shares,Asset Category Description,Primary CDS RIC
5,MAERSKb.CO,EUROPE DEVELOPED WITH FIXED INC,AP MOELLER - MAERSK A/S,1700.0,Ordinary Share,MAER5YEUAM=MT
9,CAPP.PA,EUROPE DEVELOPED WITH FIXED INC,CAPGEMINI SE,125200.0,Ordinary Share,CPGM5YEUAM=R
10,PERP.PA,EUROPE DEVELOPED WITH FIXED INC,PERNOD RICARD SA,207500.0,Ordinary Share,PERP5YEUAM=R
11,PRTP.PA,EUROPE DEVELOPED WITH FIXED INC,KERING SA,78500.0,Ordinary Share,PRTP5YEUAM=R
12,STMPA.PA,EUROPE DEVELOPED WITH FIXED INC,STMICROELECTRONICS NV,712200.0,Ordinary Share,STM5YEUAM=R
13,ALSO.PA,EUROPE DEVELOPED WITH FIXED INC,ALSTOM SA,255534.549639,Ordinary Share,ALSO5YEUAM=R
14,ADSGn.DE,EUROPE DEVELOPED WITH FIXED INC,ADIDAS AG,163600.0,Ordinary Share,ADSG5YEUAM=R
15,DBKGn.DE,EUROPE DEVELOPED WITH FIXED INC,DEUTSCHE BANK AG,797400.0,Ordinary Share,DB5YEUAM=R
16,MRCG.DE,EUROPE DEVELOPED WITH FIXED INC,MERCK KGAA,50500.0,Ordinary Share,MRCG5YEUAM=
20,STLAM.MI,EUROPE DEVELOPED WITH FIXED INC,STELLANTIS NV,940300.0,Ordinary Share,FCAU5YEUAM=R


### Price Data
At this step, we want to retrieve the snapshot price for all the Primary CDS RICs. This is the key step within our workflow as this value represents the price of the CDS instrument that we'll asses as the credit risk value. LSEG allows us to retrieve snapshot data for a list of instruments up to <b>MAX_ITEMS_PER_REQUEST</b> size. In that case, we must split the list of instruments into manageable chunks. After retrieving the snapshot data of each chunk, we'll merge the prices within our the original filtered portfolio.

In [13]:
# split a list into 'nbItems' chunks
def split_list_into_chunks(rics, nbItems):
    n = max(1, nbItems)
    return (rics[i:i+n] for i in range(0, len(rics), n))

In [18]:
# In the case our universe of RICs is very large, we'll capture the universe within manageable chunks
rics = list(assets[PRIMARY_CDS_RIC_COL])
rics_cnt = len(rics)
cds_5Y_rics_batch = split_list_into_chunks(rics, MAX_ITEMS_PER_REQUEST)

In [19]:
# Request for the pricing data for each item within our batch
#
# Note: The batch (cds_5Y_rics_batch), is a special type that returns an iterator. Once iterated, 
#       the variable has completed.  That is, you will not be able to run through the batch again.
#       If yhou wish to iterate again, you will have generate the value again by executing the 
#       cell above.
cds_5y_prices_df = pd.DataFrame()

# Simple progress bar
pb = ProgressBar(value=0, color="green")
display(pb)
print(f"Processing a total of {rics_cnt} positions...")

for items in cds_5Y_rics_batch:
    pb.value += int(min(MAX_ITEMS_PER_REQUEST, len(items))/rics_cnt * 100)  # Progress bar increment
    res = ld.get_data(universe = items, fields = [HISTORICAL_CLOSE])
    cds_5y_prices_df = cds_5y_prices_df.append(res, ignore_index=True)
print("Done")

ProgressBar(color='green', tooltip='')

Processing a total of 37 positions...
Done


In [20]:
# Display the price data for each constituent within the portfolio
cds_5y_prices_df.head(10)

,Instrument,HST_CLOSE
0,MAER5YEUAM=MT,<NA>
1,CPGM5YEUAM=R,<NA>
2,PERP5YEUAM=R,34.77
3,PRTP5YEUAM=R,49.3
4,STM5YEUAM=R,32.27
5,ALSO5YEUAM=R,83.66
6,ADSG5YEUAM=R,70.17
7,DB5YEUAM=R,58.29
8,MRCG5YEUAM=,<NA>
9,FCAU5YEUAM=R,<NA>


### Merge Data
The final step is to merge the price data within the portfolio.  In some cases, we may not have price data available for our constituents and will also filter out those unwanted values.

In [21]:
# Merge the initial dataframe with the realtime values
assets[CLOSE_PRICE_COL] = cds_5y_prices_df[HISTORICAL_CLOSE].to_list()

# Clean out any missing values
result = assets.dropna(subset=[CLOSE_PRICE_COL])

In [22]:
result.head(10)

,Instrument,Portfolio Name,Issuer Name,Portfolio Shares,Asset Category Description,Primary CDS RIC,Close Price
10,PERP.PA,EUROPE DEVELOPED WITH FIXED INC,PERNOD RICARD SA,207500.0,Ordinary Share,PERP5YEUAM=R,34.77
11,PRTP.PA,EUROPE DEVELOPED WITH FIXED INC,KERING SA,78500.0,Ordinary Share,PRTP5YEUAM=R,49.3
12,STMPA.PA,EUROPE DEVELOPED WITH FIXED INC,STMICROELECTRONICS NV,712200.0,Ordinary Share,STM5YEUAM=R,32.27
13,ALSO.PA,EUROPE DEVELOPED WITH FIXED INC,ALSTOM SA,255534.549639,Ordinary Share,ALSO5YEUAM=R,83.66
14,ADSGn.DE,EUROPE DEVELOPED WITH FIXED INC,ADIDAS AG,163600.0,Ordinary Share,ADSG5YEUAM=R,70.17
15,DBKGn.DE,EUROPE DEVELOPED WITH FIXED INC,DEUTSCHE BANK AG,797400.0,Ordinary Share,DB5YEUAM=R,58.29
22,HEIN.AS,EUROPE DEVELOPED WITH FIXED INC,HEINEKEN NV,450500.0,Ordinary Share,HEIN5YEUAM=R,30.22
25,SCAb.ST,EUROPE DEVELOPED WITH FIXED INC,SVENSKA CELLULOSA AKTIEBOLAGET SCA,480200.0,Ordinary Share,SCA5YEUAM=R,84.94
32,NXT.L,EUROPE DEVELOPED WITH FIXED INC,NEXT PLC,103500.0,Ordinary Share,NXT5YEUAM=R,58.33
34,PRU.L,EUROPE DEVELOPED WITH FIXED INC,PRUDENTIAL PLC,2148100.0,Ordinary Share,PRU5YEUAM=R,56.77


### Analysis
Now that we have successfully cleaned and priced the CDS 5Y positions, we can now use this value to derive our credit risk.  That is, the 'Close Price' value is by definition a financial derivative that allows an investor to swap or offset their credit risk with that of another investor.  With this data, it may be desirable to export the results within a simple spreadsheet for further analysis.  We perform this task next.

### Export to excel

Export the result to excel for future usage.

In [23]:
filename = f"Credit_Risk_{portfolio_id}.csv"
result.to_csv(filename)

In [24]:
# export to excel
filename = f"Credit_Risk_{portfolio_id}.xlsx"
result.to_excel(filename)

## Conclusion
While the strengths of the PAL - Portfolio & List Manager within the desktop are extremely powerful, we may come across scenarios where we need to extend the capabilities of what is available.  By coupling the features of PAL and the LSEG Data Libraries, we can easily extract the required CDS 5Y price values for a given portfolio.  By retrieving the CDS 5Y price, we can assess the credit risk exposure of the portfolio.